In [168]:
# Chuyển mô hình sang device (GPU hoặc CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

vocab_size=wordEmbeddings.shape[0] 
word_emb_dim=wordEmbeddings.shape[1] 
word_embeddings=wordEmbeddings
case_emb_dim=len(case2Idx)
case_embeddings=caseEmbeddings 
char_size=len(char2Idx)
char_emb_dim=30
conv_out_channels=30
conv_kernel_size=3
gru_hidden_size=200 
num_labels=len(label2Idx)
dropout=0.5
dropout_recurrent=0.2

# Khởi tạo mô hình (sử dụng dữ liệu embedding đã cho)
model = CNN_BGRU(
    vocab_size=vocab_size, 
    word_emb_dim=word_emb_dim, 
    word_embeddings=word_embeddings, 
    case_emb_dim=case_emb_dim,
    case_embeddings=case_embeddings, 
    char_size=char_size, 
    char_emb_dim=char_emb_dim, 
    conv_out_channels=conv_out_channels, 
    conv_kernel_size=conv_kernel_size, 
    gru_hidden_size=gru_hidden_size, 
    num_labels=num_labels,
    dropout=dropout, 
    dropout_recurrent=dropout_recurrent,
    device=device
)
model.to(device)


# Khởi tạo Optimizer và Loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()  # Loss cho classification

# Huấn luyện mô hình
train_model(model, train_loaders, dev_loaders, optimizer, criterion, epochs=10)
# train_model_baseline(model, train_loaders, dev_loaders, optimizer, criterion, epochs=30)


C:\Users\hquan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


Epoch 1/10, Loss: 0.3920
Train Precision: 0.8186, Recall: 0.7794, F1: 0.7922
Dev Precision: 0.8182, Recall: 0.7669, F1: 0.7871

Epoch 2/10, Loss: 0.2080
Train Precision: 0.8297, Recall: 0.8303, F1: 0.8239
Dev Precision: 0.8283, Recall: 0.8193, F1: 0.8176

Epoch 3/10, Loss: 0.1551
Train Precision: 0.8371, Recall: 0.8676, F1: 0.8506
Dev Precision: 0.8355, Recall: 0.8509, F1: 0.8419

Epoch 4/10, Loss: 0.1218
Train Precision: 0.8313, Recall: 0.8800, F1: 0.8521
Dev Precision: 0.8236, Recall: 0.8618, F1: 0.8396

Epoch 5/10, Loss: 0.0994
Train Precision: 0.8869, Recall: 0.8922, F1: 0.8881
Dev Precision: 0.8716, Recall: 0.8695, F1: 0.8690

Epoch 6/10, Loss: 0.0848
Train Precision: 0.9126, Recall: 0.9052, F1: 0.9075
Dev Precision: 0.8883, Recall: 0.8745, F1: 0.8801

Epoch 7/10, Loss: 0.0741
Train Precision: 0.9152, Recall: 0.9161, F1: 0.9144
Dev Precision: 0.8863, Recall: 0.8842, F1: 0.8840

Epoch 8/10, Loss: 0.0665
Train Precision: 0.9305, Recall: 0.9255, F1: 0.9270
Dev Precision: 0.8960, Reca

In [169]:
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []
    
with torch.no_grad():
    for loader in test_loaders:
        for batch in loader:
            tokens, cases, chars, labels = batch
            tokens = tokens.to(model.device)
            cases  = cases.to(model.device)
            chars  = chars.to(model.device)
            labels = labels.to(model.device)
                
            outputs = model.forward(tokens, cases, chars)  # [batch_size, seq_len, num_labels]
            preds = torch.argmax(outputs, dim=-1)  # [batch_size, seq_len]
                
            all_preds.extend(preds.cpu().numpy().tolist())
            all_labels.extend(labels.cpu().numpy().tolist())
    
y_pred = []
y_true = []
for pred_seq, true_seq in zip(all_preds, all_labels):
    for p, t in zip(pred_seq, true_seq):
            y_pred.append(p)
            y_true.append(t)
labels = list(label2Idx.values())
target_names = list(label2Idx.keys())
report = classification_report(y_true, y_pred, labels=labels, target_names=target_names, zero_division=0)
print(report)

              precision    recall  f1-score   support

       B-LOC       0.91      0.93      0.92      1668
      B-MISC       0.84      0.76      0.80       702
       B-ORG       0.83      0.89      0.86      1661
       B-PER       0.93      0.95      0.94      1617
       I-LOC       0.80      0.82      0.81       257
      I-MISC       0.56      0.61      0.58       216
       I-ORG       0.70      0.90      0.79       835
       I-PER       0.95      0.99      0.97      1156
           O       1.00      0.99      0.99     38323

    accuracy                           0.97     46435
   macro avg       0.84      0.87      0.85     46435
weighted avg       0.97      0.97      0.97     46435

